# Lab 4

## Init data

### Task and init data tast

![Task](image.png)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate as tb

X = np.array([i+1 for i in range(5)])
Y = np.array([2 + 0.5 * i for i in range(5)])

lenData = len(X)
nan = np.nan
fmt_number = '%.3f'

Data = np.array(
    [
        [nan, nan, nan, 3, 4],
        [nan, nan, 5, 3, 5],
        [nan, 3, 6, 4, nan],
        [4, 4, 2, 2, nan],
        [1, 4, nan, nan, nan]
    ]
)

ny_SumsOfYInData = np.array([np.nansum(Data[i, :]) for i in range(lenData)])
nx_SumsOfYInData = np.array([np.nansum(Data[:, i]) for i in range(lenData)])


### Init functions

#### to_strings

In [ ]:
def to_strings(a: np.ndarray):
    tp = type('')
    res = a.copy()
    match(res.ndim):
        case 1: res = np.array([str(np.round(x, 3)) for x in a], tp)
        case 2:
            for i in range(a.shape[0]):
                temp = np.array([str(np.round(x, 3)) for x in res[i]], tp)
                res[i] = temp
        case _:
            raise Exception('Unhandled ndarray dimentails amount')

    return res


#### tabulate

In [ ]:
def tabulate(a: np.ndarray, col_headers: np.ndarray = [], row_headers: np.ndarray = []):
    a_copy = a.copy()
    row_amount = a_copy.shape[0]
    if (len(row_headers) != 0 and row_amount != len(row_headers)):
        raise Exception(
            "Length of rows headers array must be equal input array rows count")
    match(len(a.shape)):
        case 1: raise Exception("Array must have more than one dimention")
        case 2:
            col_amount = a_copy.shape[1]
            colheaders_count = len(col_headers)
            if (colheaders_count > 0):
                col_amount += 1
            if (colheaders_count != 0 and col_amount != colheaders_count):
                raise Exception(
                    "Length of columns headers array must be equal input array columns count")
            if (len(row_headers) != 0):
                res = [[]
                       for i in range(row_amount)]
                for i in range(row_amount):
                    res[i].append(row_headers[i])
                    for x in to_strings(a_copy[i]):
                        res[i].append(x)
            else:
                res = a_copy
        case _:
            raise Exception('Unhandled ndarray shape situation')

    print(tb(res, headers=col_headers,
          tablefmt='rounded_grid', numalign='center'))


### Data output

In [ ]:
tabulate(Data, np.concatenate([[
    '''   \  X
Y  \\'''], to_strings(X)]), to_strings(Y))


## Correlation field

In [ ]:
# trend line
start = (np.flip(X)[0] + np.flip(Y)[0]) / 2
q = (((np.flip(X)[-1] + np.flip(Y)[-1]) / 2) - start) / lenData

tl = [start + q * i for i in range(lenData)]

notnanData = [[0, [], []] for i in range(lenData)]

for i in range(lenData):
    notnanData[i][0] = X[i]
    for k in range(lenData):
        if (np.isnan(Data[k, i])):
            continue
        notnanData[i][1].append(Y[k])
        notnanData[i][2].append(Data[k, i])

plt.plot(np.flip(Y), X, label='Y / X')
plt.plot(np.flip(X), Y, label='X / Y')
plt.plot(tl, Y, label='Trend line', c='gray', ls='--')
for i in range(lenData):
    ys = notnanData[i][1]
    plt.scatter([notnanData[i][0]
                for j in range(len(ys))], ys, c='black')
plt.title('Correlation field')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()


## Least squares regression line equation

### Plot

In [ ]:
y_avarage_xj = np.zeros(lenData)

for i in range(lenData):
    q = np.sum(notnanData[i][-1]) * Y[i]
    y_avarage_xj[i] = q / nx_SumsOfYInData[i]

plt.scatter(X, y_avarage_xj, c="black")
plt.plot(X, Y, label='X on Y')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
print('''
Видно, что отклонения точек от построенной
прямой незначительны. Следовательно, связь между признаками
X и Y может носить линейный характер
''')


### The least squares method

#### Table

In [ ]:
nxy = np.sum(nx_SumsOfYInData) + np.sum(ny_SumsOfYInData)

res = np.zeros((lenData + 6, lenData))

for i in range(lenData):
    res[i] = Data[i]
res[lenData] = nx_SumsOfYInData
res[lenData + 1] = ny_SumsOfYInData
res[lenData + 2] = [nx_SumsOfYInData[i] * X[i] for i in range(lenData)]
res[lenData + 3] = [ny_SumsOfYInData[i] * Y[i] for i in range(lenData)]
res[lenData + 4] = [nx_SumsOfYInData[i] * X[i]**2 for i in range(lenData)]
res[lenData + 5] = [np.sum([notnanData[i][0] * notnanData[i][1][j] * notnanData[i][1][j]
                            for j in range(len(notnanData[i][1]))]) for i in range(lenData)]


In [111]:
cols = np.append([
    ''' \ X
Y \\'''], to_strings(X))
rows = np.append(to_strings(
    Y), ['nx', 'ny', 'nx * x', 'ny * y', 'nx * x^2', 'nxy_xy'])

tabulate(res, cols, rows)

╭──────────┬───────┬──────┬──────┬─────┬───────╮
│  \ X     │   1   │  2   │  3   │  4  │   5   │
│ Y \      │       │      │      │     │       │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ 2.0      │  nan  │ nan  │ nan  │  3  │   4   │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ 2.5      │  nan  │ nan  │  5   │  3  │   5   │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ 3.0      │  nan  │  3   │  6   │  4  │  nan  │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ 3.5      │   4   │  4   │  2   │  2  │  nan  │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ 4.0      │   1   │  4   │ nan  │ nan │  nan  │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ nx       │   5   │  11  │  13  │ 12  │   9   │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ ny       │   7   │  13  │  13  │ 12  │   5   │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ nx * x   │   5   │  22  │  39  │ 48  │  45   │
├──────────┼───────┼──────┼──────┼─────┼───────┤
│ ny * y   │  14   │

#### The equation

In [ ]:
# TODO: the equation making (page 11)